In [1]:
import os
import sys
import numpy as np
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset, TrainCleanSoundsDataset, PsychophysicsCleanSoundsDataset

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [2]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'pnet'

# Parameters

In [3]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 50
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 70

lr = 1E-5
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
checkpoints_dir = f'{engram_dir}checkpoints/'
tensorboard_dir = f'{engram_dir}tensorboard/'
train_datafile = f'{engram_dir}clean_reconstruction_training_set.hdf5'

Device: cuda:0


In [4]:
!nvidia-smi

Tue Aug 16 18:40:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:1D:00.0 Off |                  N/A |
| 27%   26C    P8    21W / 250W |      3MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [5]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [6]:
pnet = PNetClass(net, build_graph=True)

In [7]:
pnet.eval()

PBranchedNetwork_AllSeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=

In [8]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up train/test dataset

In [9]:
train_dataset = CleanSoundsDataset(train_datafile, .9)
test_dataset = CleanSoundsDataset(train_datafile, .9, train = False)

In [10]:
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

# Set up checkpoints and tensorboards

In [11]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [12]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

/home/es3773/.conda/envs/hcnn/lib/python3.6/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


Training Epoch: 1 [50/36045]	Loss: 82736.7109
Training Epoch: 1 [100/36045]	Loss: 78279.6328
Training Epoch: 1 [150/36045]	Loss: 73610.2422
Training Epoch: 1 [200/36045]	Loss: 69165.2031
Training Epoch: 1 [250/36045]	Loss: 75245.4688
Training Epoch: 1 [300/36045]	Loss: 73900.4688
Training Epoch: 1 [350/36045]	Loss: 69926.0000
Training Epoch: 1 [400/36045]	Loss: 66059.2812
Training Epoch: 1 [450/36045]	Loss: 63671.7773
Training Epoch: 1 [500/36045]	Loss: 60593.2617
Training Epoch: 1 [550/36045]	Loss: 56903.3672
Training Epoch: 1 [600/36045]	Loss: 51955.5898
Training Epoch: 1 [650/36045]	Loss: 50205.6562
Training Epoch: 1 [700/36045]	Loss: 45935.0391
Training Epoch: 1 [750/36045]	Loss: 41558.9258
Training Epoch: 1 [800/36045]	Loss: 39320.6914
Training Epoch: 1 [850/36045]	Loss: 38060.9570
Training Epoch: 1 [900/36045]	Loss: 33600.6523
Training Epoch: 1 [950/36045]	Loss: 30438.9316
Training Epoch: 1 [1000/36045]	Loss: 31007.9414
Training Epoch: 1 [1050/36045]	Loss: 30586.0371
Training Epo

Training Epoch: 1 [8650/36045]	Loss: 12648.8457
Training Epoch: 1 [8700/36045]	Loss: 12898.6299
Training Epoch: 1 [8750/36045]	Loss: 12966.9814
Training Epoch: 1 [8800/36045]	Loss: 12886.6133
Training Epoch: 1 [8850/36045]	Loss: 12926.1797
Training Epoch: 1 [8900/36045]	Loss: 11865.7529
Training Epoch: 1 [8950/36045]	Loss: 11915.1445
Training Epoch: 1 [9000/36045]	Loss: 12267.1006
Training Epoch: 1 [9050/36045]	Loss: 12368.3799
Training Epoch: 1 [9100/36045]	Loss: 12250.6885
Training Epoch: 1 [9150/36045]	Loss: 8444.8945
Training Epoch: 1 [9200/36045]	Loss: 5920.9644
Training Epoch: 1 [9250/36045]	Loss: 6413.6704
Training Epoch: 1 [9300/36045]	Loss: 6657.3643
Training Epoch: 1 [9350/36045]	Loss: 5912.9863
Training Epoch: 1 [9400/36045]	Loss: 12149.0547
Training Epoch: 1 [9450/36045]	Loss: 12719.0029
Training Epoch: 1 [9500/36045]	Loss: 12443.7617
Training Epoch: 1 [9550/36045]	Loss: 12975.6104
Training Epoch: 1 [9600/36045]	Loss: 9593.9209
Training Epoch: 1 [9650/36045]	Loss: 9065.7158

Training Epoch: 1 [17150/36045]	Loss: 7668.8857
Training Epoch: 1 [17200/36045]	Loss: 8256.3252
Training Epoch: 1 [17250/36045]	Loss: 8460.4502
Training Epoch: 1 [17300/36045]	Loss: 8833.4268
Training Epoch: 1 [17350/36045]	Loss: 7568.2695
Training Epoch: 1 [17400/36045]	Loss: 7355.5757
Training Epoch: 1 [17450/36045]	Loss: 7266.1738
Training Epoch: 1 [17500/36045]	Loss: 7098.9111
Training Epoch: 1 [17550/36045]	Loss: 7239.1631
Training Epoch: 1 [17600/36045]	Loss: 6958.1885
Training Epoch: 1 [17650/36045]	Loss: 7172.1357
Training Epoch: 1 [17700/36045]	Loss: 6800.3706
Training Epoch: 1 [17750/36045]	Loss: 7102.8706
Training Epoch: 1 [17800/36045]	Loss: 6928.6035
Training Epoch: 1 [17850/36045]	Loss: 6297.7661
Training Epoch: 1 [17900/36045]	Loss: 6402.6870
Training Epoch: 1 [17950/36045]	Loss: 6251.4019
Training Epoch: 1 [18000/36045]	Loss: 6194.7168
Training Epoch: 1 [18050/36045]	Loss: 7780.9214
Training Epoch: 1 [18100/36045]	Loss: 8124.3794
Training Epoch: 1 [18150/36045]	Loss: 80

Training Epoch: 1 [25700/36045]	Loss: 7507.5098
Training Epoch: 1 [25750/36045]	Loss: 6855.3271
Training Epoch: 1 [25800/36045]	Loss: 3549.5154
Training Epoch: 1 [25850/36045]	Loss: 3381.6843
Training Epoch: 1 [25900/36045]	Loss: 3285.3953
Training Epoch: 1 [25950/36045]	Loss: 3321.1597
Training Epoch: 1 [26000/36045]	Loss: 4200.0815
Training Epoch: 1 [26050/36045]	Loss: 5998.1694
Training Epoch: 1 [26100/36045]	Loss: 5902.2954
Training Epoch: 1 [26150/36045]	Loss: 5871.8999
Training Epoch: 1 [26200/36045]	Loss: 5760.1812
Training Epoch: 1 [26250/36045]	Loss: 5870.8643
Training Epoch: 1 [26300/36045]	Loss: 4885.3589
Training Epoch: 1 [26350/36045]	Loss: 4763.1948
Training Epoch: 1 [26400/36045]	Loss: 4787.4053
Training Epoch: 1 [26450/36045]	Loss: 4662.1191
Training Epoch: 1 [26500/36045]	Loss: 5913.9458
Training Epoch: 1 [26550/36045]	Loss: 6142.2002
Training Epoch: 1 [26600/36045]	Loss: 6157.4907
Training Epoch: 1 [26650/36045]	Loss: 6186.4702
Training Epoch: 1 [26700/36045]	Loss: 61

...

Training Epoch: 51 [21150/36045]	Loss: 515.4752
Training Epoch: 51 [21200/36045]	Loss: 514.4052
Training Epoch: 51 [21250/36045]	Loss: 491.8866
Training Epoch: 51 [21300/36045]	Loss: 573.9794
Training Epoch: 51 [21350/36045]	Loss: 565.7933
Training Epoch: 51 [21400/36045]	Loss: 569.5728
Training Epoch: 51 [21450/36045]	Loss: 575.7029
Training Epoch: 51 [21500/36045]	Loss: 577.0111
Training Epoch: 51 [21550/36045]	Loss: 671.0237
Training Epoch: 51 [21600/36045]	Loss: 669.3292
Training Epoch: 51 [21650/36045]	Loss: 681.6258
Training Epoch: 51 [21700/36045]	Loss: 685.2303
Training Epoch: 51 [21750/36045]	Loss: 657.5723
Training Epoch: 51 [21800/36045]	Loss: 481.8662
Training Epoch: 51 [21850/36045]	Loss: 464.8820
Training Epoch: 51 [21900/36045]	Loss: 474.0473
Training Epoch: 51 [21950/36045]	Loss: 475.9550
Training Epoch: 51 [22000/36045]	Loss: 479.1029
Training Epoch: 51 [22050/36045]	Loss: 497.6655
Training Epoch: 51 [22100/36045]	Loss: 490.0591
Training Epoch: 51 [22150/36045]	Loss: 4

Training Epoch: 51 [29700/36045]	Loss: 416.0654
Training Epoch: 51 [29750/36045]	Loss: 414.1762
Training Epoch: 51 [29800/36045]	Loss: 460.7945
Training Epoch: 51 [29850/36045]	Loss: 538.7386
Training Epoch: 51 [29900/36045]	Loss: 534.0318
Training Epoch: 51 [29950/36045]	Loss: 556.0562
Training Epoch: 51 [30000/36045]	Loss: 529.8306
Training Epoch: 51 [30050/36045]	Loss: 537.2250
Training Epoch: 51 [30100/36045]	Loss: 654.8904
Training Epoch: 51 [30150/36045]	Loss: 636.7601
Training Epoch: 51 [30200/36045]	Loss: 600.5110
Training Epoch: 51 [30250/36045]	Loss: 649.0861
Training Epoch: 51 [30300/36045]	Loss: 633.0320
Training Epoch: 51 [30350/36045]	Loss: 479.7457
Training Epoch: 51 [30400/36045]	Loss: 464.6039
Training Epoch: 51 [30450/36045]	Loss: 467.0789
Training Epoch: 51 [30500/36045]	Loss: 436.0144
Training Epoch: 51 [30550/36045]	Loss: 404.7924
Training Epoch: 51 [30600/36045]	Loss: 398.1963
Training Epoch: 51 [30650/36045]	Loss: 388.4057
Training Epoch: 51 [30700/36045]	Loss: 4

Training Epoch: 52 [2250/36045]	Loss: 476.5611
Training Epoch: 52 [2300/36045]	Loss: 499.8396
Training Epoch: 52 [2350/36045]	Loss: 477.9295
Training Epoch: 52 [2400/36045]	Loss: 485.1205
Training Epoch: 52 [2450/36045]	Loss: 621.8163
Training Epoch: 52 [2500/36045]	Loss: 652.8709
Training Epoch: 52 [2550/36045]	Loss: 651.0138
Training Epoch: 52 [2600/36045]	Loss: 659.8886
Training Epoch: 52 [2650/36045]	Loss: 783.7657
Training Epoch: 52 [2700/36045]	Loss: 871.8410
Training Epoch: 52 [2750/36045]	Loss: 942.2822
Training Epoch: 52 [2800/36045]	Loss: 951.3830
Training Epoch: 52 [2850/36045]	Loss: 718.1073
Training Epoch: 52 [2900/36045]	Loss: 678.4183
Training Epoch: 52 [2950/36045]	Loss: 656.6024
Training Epoch: 52 [3000/36045]	Loss: 650.2089
Training Epoch: 52 [3050/36045]	Loss: 680.6914
Training Epoch: 52 [3100/36045]	Loss: 621.5785
Training Epoch: 52 [3150/36045]	Loss: 477.5189
Training Epoch: 52 [3200/36045]	Loss: 494.3625
Training Epoch: 52 [3250/36045]	Loss: 466.9685
Training Epoc

Training Epoch: 52 [10950/36045]	Loss: 490.2926
Training Epoch: 52 [11000/36045]	Loss: 484.1413
Training Epoch: 52 [11050/36045]	Loss: 519.2975
Training Epoch: 52 [11100/36045]	Loss: 529.1519
Training Epoch: 52 [11150/36045]	Loss: 574.1995
Training Epoch: 52 [11200/36045]	Loss: 603.0364
Training Epoch: 52 [11250/36045]	Loss: 614.0839
Training Epoch: 52 [11300/36045]	Loss: 594.5507
Training Epoch: 52 [11350/36045]	Loss: 592.3343
Training Epoch: 52 [11400/36045]	Loss: 556.2299
Training Epoch: 52 [11450/36045]	Loss: 525.7998
Training Epoch: 52 [11500/36045]	Loss: 523.4216
Training Epoch: 52 [11550/36045]	Loss: 532.6005
Training Epoch: 52 [11600/36045]	Loss: 592.1937
Training Epoch: 52 [11650/36045]	Loss: 643.1219
Training Epoch: 52 [11700/36045]	Loss: 642.0696
Training Epoch: 52 [11750/36045]	Loss: 659.8582
Training Epoch: 52 [11800/36045]	Loss: 702.2770
Training Epoch: 52 [11850/36045]	Loss: 760.9034
Training Epoch: 52 [11900/36045]	Loss: 972.1187
Training Epoch: 52 [11950/36045]	Loss: 9

KeyboardInterrupt: 